<a href="https://colab.research.google.com/github/Kim-Dong-Jun99/DSTermProject/blob/kim-dong-jun99/term_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#공유 드라이브 연동

from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive', force_remount = True)  # 공유 드라이브

df_2015_path = r'/content/gdrive/Shareddrives/data_science/dataset/2015.csv'
df_2016_path = r'/content/gdrive/Shareddrives/data_science/dataset/2016.csv'
df_2017_path = r'/content/gdrive/Shareddrives/data_science/dataset/2017.csv'


In [ ]:
df_2015 = pd.read_csv(df_2015_path)
df_2016 = pd.read_csv(df_2016_path)
df_2017 = pd.read_csv(df_2017_path)

# TermProject

## Source and explanation of the dataset

The dataset is 'World Happiness Report' from Kaggle. 'World Happiness Report' has data of world happiness from 2015 to 2019

https://www.kaggle.com/datasets/unsdsn/world-happiness?datasetId=894&sortBy=voteCount&searchQuery=predict

The columns of each dataset include 'Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Economy', 'Family', 'Health', 'Freedom', 'Trust', 'Generosity', 'Dystopia Residual' etc.

It is arranged in descending order according to the happiness score. The happiness score is the sum of the scores of other numerical features (except for Hapiness Rank).

The World Happiness Report is a landmark survey of the state of global happiness. The reports review the state of happiness in the world today and show how the


# Objective of our analysis
Analyze dataset to figure out characteristics of each continent in dataset then predict continent value of countries in dataset of 2017 using 2015, 2016 dataset for training

We will use clustering and classification algorithm to predict continent of countries in dataset of 2017 using 2015, 2016 happiness data report

Also, we will automate procedure of choosing best combination of scaler and algorithm that gives best result